## Clip Dynamic GT and nuScenes

In [75]:
import cv2
import numpy as np
import os

In [ ]:
import glob

type = 'static'
# type = 'dynamic'
model_input_folder = '/data/nuscenes/nuscenes_mini/samples/CAM_FRONT'
model_output_folder = '/home/pak/vkr'
model_gt_folder = f'/home/pak/vkr/data/datasets/nuscenes/nuscenes_mini/{type}_gt'

In [ ]:
sorted_input_names = sorted(glob.glob(f'{model_input_folder}/**.jpg', recursive=True))

image_sample = cv2.imread(sorted_input_names[0])

height = image_sample.shape[0]
width = image_sample.shape[0]

size = (image_sample.shape[1] + image_sample.shape[0], image_sample.shape[0])

In [ ]:
fps = 5
out = cv2.VideoWriter(model_output_folder + '/fetched.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for img_index, img_path in enumerate(sorted_input_names):    
    input_image = cv2.imread(img_path)
    gt_image = cv2.imread(img_path.replace('samples/CAM_FRONT', f'{type}_gt'))
        
    gt_image = cv2.resize(
        gt_image, dsize=(height, width), interpolation=cv2.INTER_CUBIC
    )
    data = np.concatenate((input_image, gt_image), 1)
    out.write(data)
out.release()

## Clip static+dynamic video

In [ ]:
sorted_input_names = sorted(glob.glob(f'{model_input_folder}/**.jpg', recursive=True))
sorted_static_names = sorted(glob.glob(f'{model_output_folder}/static/**.png', recursive=True))

image_sample = cv2.imread(sorted_input_names[0])
static_sample = cv2.imread(sorted_static_names[0])
dynamic_sample = cv2.imread(sorted_static_names[0].replace('static', 'dynamic'))

height = image_sample.shape[0]
width = image_sample.shape[0]

size = (image_sample.shape[1] + image_sample.shape[0], image_sample.shape[0])

In [ ]:
fps = 5
out = cv2.VideoWriter(model_output_folder + '/fetched.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for img_index, img_path in enumerate(sorted_static_names):
    assert img_path.split('_')[-1][:-3] == sorted_input_names[img_index].split('_')[-1][:-3]
    
    input_image = cv2.imread(sorted_input_names[img_index])
    static_image = cv2.imread(img_path)
    dynamic_image = cv2.imread(img_path.replace('static', 'dynamic'))
    
    dynamic_image[dynamic_image == 255] = 128
    
    static_image = cv2.resize(
        static_image, dsize=(height, width), interpolation=cv2.INTER_CUBIC
    )
    dynamic_image = cv2.resize(
        dynamic_image, dsize=(height, width), interpolation=cv2.INTER_CUBIC
    )
    data = np.concatenate((input_image, dynamic_image + static_image), 1)
    out.write(data)
out.release()

## Make train-val split

In [ ]:
import os
import glob

In [ ]:
model_gt_folder = '/home/pak/vkr/data/datasets/nuscenes/nuscenes_mini/dynamic_gt'

arr = sorted(glob.glob(f'{model_gt_folder}/**.jpg', recursive=True))

In [ ]:
arr1 = ['static' + x[58:] for x in arr]

In [ ]:
with open('/home/pak/vkr/monolayout/splits/nuscenes/train_files.txt', 'w') as outfile:
    for x in arr1:
        outfile.write(x)
        outfile.write('\n')

with open('/home/pak/vkr/monolayout/splits/nuscenes/valid_files.txt', 'w') as outfile:
    for x in arr1:
        outfile.write(x)
        outfile.write('\n')

## Get Static GT

In [80]:
%pip install nuscenes-devkit

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 283 kB 3.6 MB/s 
     |████████████████████████████████| 87 kB 4.8 MB/s 
     |████████████████████████████████| 1.0 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 13.1 MB/s 
     |████████████████████████████████| 9.4 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 74.0 MB/s 
     |████████████████████████████████| 780 kB 74.1 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=273255 sha256=f8872ed71e45058599e891ed2871dda4ebe93b6527210c3c17e5e90e40ba5788
  Stored in directory: /home/jupyter/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=68b7856011f7c6fd3c1ab39b89c3c358c21a03112f9ddc4b35b6be8ebea

In [83]:
%pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 29.2 MB 33.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 72.2 MB/s 
     |████████████████████████████████| 3.3 MB 68.4 MB/s 
     |████████████████████████████████| 165 kB 81.5 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.


In [84]:
import argparse
import os

from matplotlib import pyplot as plt

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import transform_matrix, Quaternion

from nuscenes.map_expansion.map_api import NuScenesMap
from nuscenes.map_expansion import arcline_path_utils
from nuscenes.map_expansion.bitmap import BitMap

import numpy as np

from PIL import Image
from skimage.draw import polygon


In [90]:
data_root = '/home/jupyter/work/resources/data/nuscenes/'
nusc_version = 'v1.0-mini'

nusc = NuScenes(version=nusc_version, dataroot=data_root, verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.650 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [91]:
def get_mask_angle(ego_pose, cam_pose):
    cam2ego = transform_matrix(cam_pose['translation'], Quaternion(cam_pose['rotation']), inverse=False)
    ego2world = transform_matrix(ego_pose['translation'], Quaternion(ego_pose['rotation']), inverse=False)
    cam2world = np.dot(ego2world, cam2ego)

    rel_unit = np.dot(cam2world, [0, 0, 1, 0])
    angle = 180 -180*np.arctan2(rel_unit[0], rel_unit[1]) / np.pi

    return angle


In [94]:
occ_width, occ_height = 256, 256
sensor = 'CAM_FRONT'

output_dir = f'{data_root}/static_gt'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

old_location = None

for sample in nusc.sample:
    my_sample = nusc.get('sample', sample['token'])

    cam_front_data = nusc.get('sample_data', my_sample['data'][sensor])
    ego_pose = nusc.get('ego_pose', cam_front_data['ego_pose_token'])
    cam_pose = nusc.get('calibrated_sensor', cam_front_data['calibrated_sensor_token'])
    scene = nusc.get('scene', my_sample['scene_token'])
    log = nusc.get('log', scene['log_token'])

    if old_location != log['location']:
        nusc_map = NuScenesMap(dataroot=data_root, map_name=log['location'])
        old_location = log['location']

    scale = 32

    patch_box = [
        ego_pose['translation'][0],
        ego_pose['translation'][1],
        2 * scale,
        2 * scale
    ]
    patch_angle = get_mask_angle(ego_pose, cam_pose)
    layer_names = ['drivable_area']


    map_mask = nusc_map.get_map_mask(patch_box, patch_angle, layer_names, (2 * occ_width, 2 * occ_height))[0]
    map_mask = map_mask[:occ_height, int(occ_width / 2): -int((occ_width + 1) / 2)]

    map_mask = np.flip(map_mask, 1)

    img = Image.fromarray(map_mask * 255)
#     img.show()
    # nusc.render_sample_data(cam_front_data['token'])
    img.save(os.path.join(output_dir, cam_front_data['filename'].split('/')[-1]))

    # break

/kernel/lib/python3.7/site-packages/ml_kernel/ignored_keyboard_interrupt.py:16: UserWarning: State committing stage cannot be interrupted. Please wait.
  warnings.warn(self._warn_message)


In [ ]:
def get_fov_mask(cam_data):
    occ_width, occ_height = 256, 256

    print(cam_front_data)

    sensor_data = nusc.get('calibrated_sensor', cam_front_data['calibrated_sensor_token'])

    filename = cam_front_data['filename']
    img = Image.open(f'{data_root}{filename}')

    f_x = sensor_data['camera_intrinsic'][0][0]
    h = img.size[0]

    fov_x = 2 * np.arctan(h / (2 * f_x))
    alpha = (np.pi - fov_x) / 2

    polygons = [
        [0, occ_height - occ_width * np.tan(alpha) / 2],
        [occ_width / 2, occ_height],
        [occ_width, occ_height - occ_width * np.tan(alpha) / 2],
        [occ_width, occ_height],
        [0, occ_height]
    ]
    
    polygons = np.array(polygons)

    rr, cc = polygon(polygons[:, 1], polygons[:, 0], (occ_width, occ_height))
    ans = 255 * np.ones((occ_width, occ_height), dtype='uint8')
    ans[rr, cc] = 0

    return ans

In [ ]:
occ_width, occ_height = 256, 256
sensor = 'CAM_FRONT'

output_dir = f'{data_root}/static_gt'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

fov_mask = None

for sample in nusc.sample:
    my_sample = nusc.get('sample', sample['token'])

    cam_front_data = nusc.get('sample_data', my_sample['data'][sensor])
    ego_pose = nusc.get('ego_pose', cam_front_data['ego_pose_token'])
    cam_pose = nusc.get('calibrated_sensor', cam_front_data['calibrated_sensor_token'])
    scene = nusc.get('scene', my_sample['scene_token'])
    log = nusc.get('log', scene['log_token'])

    if fov_mask == None:
        fov_mask = get_fov_mask(cam_front_data)

        img = Image.fromarray(fov_mask)
        img.show()

    print(img.size)
    break

In [ ]:
fov_x = 2 * np.arctan( 1600 / (2 * nusc.get('calibrated_sensor', '1d31c729b073425e8e0202c5c6e66ee1')['camera_intrinsic'][0][0]))
print(fov_x, 180 * fov_x / np.pi)